In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
import json


# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")


# Defining Agent's node
##################################################################################

# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Node
def chatbot(state: MessagesState):
   return {"messages": [llm.invoke([chatbot_system_message] + state["messages"])]}


# Defining Graph
##################################################################################

builder = StateGraph(MessagesState)
builder.add_node("chatbot", chatbot)


# Define edges: these determine how the control flow moves
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
chatbot_graph = builder.compile(checkpointer=memory)

# Show
display(Image(chatbot_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
thread_id = "2"

config = {"configurable": {"thread_id": thread_id}}

chatbot_graph.invoke({"messages": [HumanMessage(content="Hi, I’m working on a Python project, and I’m stuck with handling API responses.")]}, config)

In [ ]:
chatbot_graph.invoke({"messages": [HumanMessage(content="Sorry what was my previous question?")]}, config)

In [ ]:
chatbot_graph.invoke({"messages": [HumanMessage(content="Ahh, yeah right! So I’m mostly struggling with parsing JSON responses. Sometimes the structure isn’t what I expect, and it breaks my code.")]}, config)

In [ ]:
chatbot_graph.invoke({"messages": [HumanMessage(content="Got it! That helps a lot. What would be the best way to handle deeply nested JSON data when I only need a few specific values?")]}, config)

# Defining your own schema

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")

# Defining Schema
##################################################################################
class State(MessagesState):
    question: str
    answer: str


# Defining Agent's node
##################################################################################

# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Node
def chatbot(state: State) -> State:
    question = HumanMessage(content=state.get("question", ""))
    response = llm.invoke([chatbot_system_message] + state["messages"] + [question]);

    return State(
        messages = [question, response],
        question = state.get("question", None),
        answer = response.content
    )


# Defining Graph
##################################################################################

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)


# Define edges: these determine how the control flow moves
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
chatbot_graph = builder.compile(checkpointer=memory)

# Show
display(Image(chatbot_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
thread_id = "2"

config = {"configurable": {"thread_id": thread_id}}

chatbot_graph.invoke(State(question="Hi, I’m working on a Python project, and I’m stuck with handling API responses."), config)

In [ ]:
chatbot_graph.invoke(State(question="Sorry what was my previous question?"), config)

In [ ]:
chatbot_graph.invoke(State(question="Ahh, yeah right! So I’m mostly struggling with parsing JSON responses. Sometimes the structure isn’t what I expect, and it breaks my code."), config)

In [ ]:
chatbot_graph.invoke(State(question="Got it! That helps a lot. What would be the best way to handle deeply nested JSON data when I only need a few specific values?"), config)

# Message Shrinking

## Limit number of messages

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")

# Defining Schema
##################################################################################
class State(MessagesState):
    question: str
    answer: str


# Defining Agent's node
##################################################################################

# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Node
def chatbot(state: State) -> State:
    question = HumanMessage(content=state.get("question", ""))

    trimed_messages = (state["messages"] + [question])[-4:]
    print(f"Messages sent to LLM: {len(trimed_messages)}")

    response = llm.invoke([chatbot_system_message] + trimed_messages);

    return State(
        messages = [question, response],
        question = state.get("question", None),
        answer = response.content
    )


# Defining Graph
##################################################################################

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)


# Define edges: these determine how the control flow moves
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
chatbot_graph = builder.compile(checkpointer=memory)

# Show
display(Image(chatbot_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
thread_id = "1"

config = {"configurable": {"thread_id": thread_id}}

chatbot_graph.invoke(State(question="Hi, I’m working on a Python project, and I’m stuck with handling API responses."), config)

In [ ]:
chatbot_graph.invoke(State(question="Sorry what was my previous question?"), config)

In [ ]:
chatbot_graph.invoke(State(question="Ahh, yeah right! So I’m mostly struggling with parsing JSON responses. Sometimes the structure isn’t what I expect, and it breaks my code."), config)

## Trim messages

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from langchain_core.messages import trim_messages

# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")

# Defining Schema
##################################################################################
class State(MessagesState):
    question: str
    answer: str


# Defining Agent's node
##################################################################################

# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Node
def chatbot(state: State) -> State:
    question = HumanMessage(content=state.get("question", ""))

    trimed_messages = trim_messages(
            state["messages"] + [question],
            max_tokens=200,                # keep 4-5 messages including longer detailed responses
            strategy="last",                # keep the most recent
            token_counter=llm,              # the model used to count tokens
            allow_partial=False             # only include complete messages (preserve message integrity)
        )
    print(f"Messages sent to LLM: {len(trimed_messages)}")

    response = llm.invoke([chatbot_system_message] + trimed_messages);

    return State(
        messages = [question, response],
        question = state.get("question", None),
        answer = response.content
    )@


# Defining Graph
##################################################################################

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)


# Define edges: these determine how the control flow moves
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
chatbot_graph = builder.compile(checkpointer=memory)

# Show
display(Image(chatbot_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
thread_id = "3"

config = {"configurable": {"thread_id": thread_id}}

chatbot_graph.invoke(State(question="Hi, I’m working on a Python project, and I’m stuck with handling API responses."), config)

In [ ]:
chatbot_graph.invoke(State(question="Sorry what was my previous question?"), config)

In [ ]:
chatbot_graph.invoke(State(question="Ahh, yeah right! So I’m mostly struggling with parsing JSON responses. Sometimes the structure isn’t what I expect, and it breaks my code."), config)

In [ ]:
chatbot_graph.invoke(State(question="Got it! That helps a lot. What would be the best way to handle deeply nested JSON data when I only need a few specific values?"), config)

let's allow partial!